In [1]:
import glob
train_files=glob.glob("meta_csv/train/anomaly/*.csv")
train_files

['meta_csv/train/anomaly\\0.csv',
 'meta_csv/train/anomaly\\2.csv',
 'meta_csv/train/anomaly\\3.csv',
 'meta_csv/train/anomaly\\4.csv',
 'meta_csv/train/anomaly\\5.csv']

In [2]:
import pandas as pd
import numpy as np
dfs_train=[pd.read_csv(pth) for pth in train_files]

In [3]:
dfs_train[0]

,time_pos,byte_pos,size,type,normal,has_blur,has_move,has_overexposure,has_block
0,0.00,0,0.437067,I,1,0,0,0,0
1,0.04,113467,0.011355,P,1,0,0,0,0
2,0.08,116415,0.013570,P,1,0,0,0,0
3,0.12,119938,0.008848,P,1,0,0,0,0
4,0.16,122235,0.007916,P,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
18819,752.76,229688194,0.015955,P,1,0,0,0,0
18820,752.80,229692336,0.014915,P,1,0,0,0,0
18821,752.84,229696208,0.014329,P,1,0,0,0,0
18822,752.88,229699928,0.012788,P,1,0,0,0,0


In [4]:
Xs_train,ys_train=[],[]
X_val,y_val=None,None
labels=['normal', 'has_blur', 'has_move', 'has_overexposure', 'has_block']
features=["size","type"]
for i,df in enumerate(dfs_train):
    print(df.isna().sum())
    if i==5-1:
        X_val,y_val=df[features],df[labels]
    else:
        Xs_train.append(df[features])
        ys_train.append(df[labels].values)

time_pos            0
byte_pos            0
size                0
type                0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
time_pos            0
byte_pos            0
size                0
type                0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
time_pos            0
byte_pos            0
size                0
type                0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
time_pos            0
byte_pos            0
size                0
type                0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
time_pos            0
byte_pos            0
size                0
type                0
normal              0
has_blur            0
has_move            0
ha

In [5]:
Xs_train[0]

,size,type
0,0.437067,I
1,0.011355,P
2,0.013570,P
3,0.008848,P
4,0.007916,P
...,...,...
18819,0.015955,P
18820,0.014915,P
18821,0.014329,P
18822,0.012788,P


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

In [7]:
pipeline=ColumnTransformer([
    ("num",StandardScaler(),["size"]),
    ("cat",OneHotEncoder(drop="first"),["type"])])

def reshape_data(data,timesteps):

    reshaped_data=[]
    for d in data:
        d_take=d[:d.shape[0]//timesteps*timesteps]
        reshaped_data.append(d_take.reshape(-1,timesteps,d_take.shape[1]))
    return reshaped_data
def pipeline_on_array(arr,pipeline):
    results=[]
    try:
        for data in arr:
            results.append(pipeline.transform(data))
    except:
        
        results.append(pipeline.fit_transform(arr[0]))
        for data in arr[1:]:
            results.append(pipeline.transform(data))
            print(data.shape,results[-1].shape)
    return results
def full_pipeline(Xs,ys,pipeline,timesteps):
    Xs_new=reshape_data(pipeline_on_array(Xs,pipeline),timesteps)
    ys_new=reshape_data(ys,timesteps)
    return Xs_new,ys_new

In [43]:
timesteps=1
Xs_train_rdy,ys_train_rdy=full_pipeline(Xs_train,ys_train,pipeline,timesteps)
X_val_rdy=full_pipeline([X_val],[],pipeline,timesteps)[0][0]
y_val_rdy=full_pipeline([],[y_val.values],pipeline,timesteps)[1][0]
y_val_rdy,X_val_rdy

(array([[[1, 0, 0, 0, 0]],
 
        [[1, 0, 0, 0, 0]],
 
        [[1, 0, 0, 0, 0]],
 
        ...,
 
        [[1, 0, 0, 0, 0]],
 
        [[1, 0, 0, 0, 0]],
 
        [[1, 0, 0, 0, 0]]], dtype=int64),
 array([[[ 7.50063737,  1.        ,  0.        ]],
 
        [[-0.35790031,  0.        ,  1.        ]],
 
        [[-0.12961791,  0.        ,  1.        ]],
 
        ...,
 
        [[-0.49491171,  0.        ,  1.        ]],
 
        [[-0.51148737,  0.        ,  1.        ]],
 
        [[-0.52764338,  0.        ,  1.        ]]]))

In [51]:
from tensorflow import keras
model=keras.Sequential([
    keras.layers.LSTM(32, input_shape=(timesteps,3),activation="tanh",return_sequences=True),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(5,activation="softmax")
])
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 1, 32)             4608      
                                                                 
 dropout_13 (Dropout)        (None, 1, 32)             0         
                                                                 
 dense_14 (Dense)            (None, 1, 5)              165       
                                                                 
Total params: 4773 (18.64 KB)
Trainable params: 4773 (18.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
for X_train,y_train in zip(Xs_train_rdy,ys_train_rdy):
    print(X_train.shape,y_train.shape)
    model.fit(X_train,y_train
              ,shuffle=False,
              validation_data=(X_val_rdy,y_val_rdy),batch_size=32,epochs=10)

(18824, 1, 3) (18824, 1, 5)
Epoch 1/10
589/589 [==============================] - 2s 2ms/step - loss: 0.5655 - accuracy: 0.9369 - val_loss: 1.4771 - val_accuracy: 0.6596
Epoch 2/10
589/589 [==============================] - 1s 2ms/step - loss: 0.2242 - accuracy: 0.9569 - val_loss: 1.5414 - val_accuracy: 0.6596
Epoch 3/10
589/589 [==============================] - 1s 2ms/step - loss: 0.2039 - accuracy: 0.9569 - val_loss: 1.5892 - val_accuracy: 0.6596
Epoch 4/10
589/589 [==============================] - 1s 2ms/step - loss: 0.1974 - accuracy: 0.9569 - val_loss: 1.6395 - val_accuracy: 0.6596
Epoch 5/10
589/589 [==============================] - 1s 2ms/step - loss: 0.1909 - accuracy: 0.9569 - val_loss: 1.6926 - val_accuracy: 0.6596
Epoch 6/10
589/589 [==============================] - 1s 2ms/step - loss: 0.1876 - accuracy: 0.9569 - val_loss: 1.7520 - val_accuracy: 0.6596
Epoch 7/10
589/589 [==============================] - 1s 2ms/step - loss: 0.1861 - accuracy: 0.9569 - val_loss: 1.7977 -

In [ ]:
loss,accuracy=model.evaluate(X_val_rdy,y_val_rdy)